In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
) 

80142336/80134624 [==============================] - 9s 0us/step


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [ ]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 28s - loss: 0.6064 - auc: 0.7341 - accuracy: 0.6715 - val_loss: 0.5936 - val_auc: 0.8991 - val_accuracy: 0.6484
Epoch 2/30
40/40 - 27s - loss: 0.4298 - auc: 0.8916 - accuracy: 0.8086 - val_loss: 0.3785 - val_auc: 0.9346 - val_accuracy: 0.8255
Epoch 3/30
40/40 - 27s - loss: 0.3538 - auc: 0.9380 - accuracy: 0.8637 - val_loss: 0.3442 - val_auc: 0.9628 - val_accuracy: 0.8594
Epoch 4/30
40/40 - 27s - loss: 0.2980 - auc: 0.9671 - accuracy: 0.9000 - val_loss: 0.3129 - val_auc: 0.9790 - val_accuracy: 0.8854
Epoch 5/30
40/40 - 26s - loss: 0.2672 - auc: 0.9735 - accuracy: 0.9157 - val_loss: 0.3260 - val_auc: 0.9859 - val_accuracy: 0.8542
Epoch 6/30
40/40 - 26s - loss: 0.2395 - auc: 0.9817 - accuracy: 0.9287 - val_loss: 0.2202 - val_auc: 0.9913 - val_accuracy: 0.9401
Epoch 7/30
40/40 - 26s - loss: 0.2179 - auc: 0.9858 - accuracy: 0.9354 - val_loss: 0.2276 - val_auc: 0.9939 - val_accuracy: 0.9375
Epoch 8/30
40/40 - 26s - loss: 0.1924 - auc: 0.9916 - accuracy: 0.9508 - val_loss: 

In [ ]:
prediction = model.predict(valid_generator)

In [ ]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

In [ ]:
prediction

In [ ]:
tf.__version__

In [ ]:
model.evaluate(valid_generator)